In [1]:
%run '/home/lena/Desktop/Application Project/src/ibm1.py'
%run '/home/lena/Desktop/Application Project/src/ibm2.py'
%run '/home/lena/Desktop/Application Project/src/pbmt.py'
%run '/home/lena/Desktop/Application Project/src/utils.py'
%run '/home/lena/Desktop/Application Project/src/test.py'
import numpy as np

In [2]:
TEST = True 
nr_used_sentences = 100 #1800 for 1model,25steps  # of 688670 lines
path = "./src/Paralel Corpus/"
e, f = read_in_corpus(nr_used_sentences, path)

training_split = 0.9
e_train, e_test = split_dataset(e, training_split)
f_train, f_test = split_dataset(f, training_split)

if TEST:
    e_train, f_train, e_test, f_test, all_alignments_test = initialize_test_sets()

max_steps = 25

t_e2f_ibm1 = EM_IBM_Model_1(e_train, f_train, max_steps)
t_e2f, a_e2f = EM_IBM_Model_2(e_train, f_train, t_e2f_ibm1, max_steps)
print('------')
t_f2e_ibm1 = EM_IBM_Model_1(f_train, e_train, max_steps)
t_f2e, a_f2e = EM_IBM_Model_2(f_train, e_train, t_f2e_ibm1, max_steps)

#if TEST:
    #compare_ibm_1(t_e2f, max_steps, e_train, f_train)
    #compare_ibm_2(t, max_steps, a, e_train, f_train)
    #compare_ibm_2(t, max_steps, a, f_train, e_train)

sum = 0
for i in range(len(f_test)):
    if (i,4,2,2) in a_f2e:
        sum += a[(i,4,2,2)]
   # else:
    #    print('tüdelü')
#print(sum,f_test[i],e_test[4],' TODO sum is supposed to always be 1')

# Testing
#TODO p(e|f) calc is wrong!!!!
wrong1 = 0
wrong2 = 0
for i in range(0, len(f_test)):
    f = f_test[i]
    e = e_test[i]
    p1 = prob_e_given_f_1(e, f, 1, t_e2f)
    if (p1 < 0.01):
        wrong1 += 1
    print('p(', e, '|', f, ') =', p1)
    p2 = 0#prob_e_given_f_2(e, f, 1, t, a)
    if (p2 < 0.5):
        wrong2 += 1
    #print('a: p(', e, '|', f, ') =', p2)
    
print('IBM Model 1 :', int(100*(1-wrong1/len(f_test))),'% correct')
print('IBM Model 2 :', int(100*(1-wrong2/len(f_test))),'% correct')
        
# arg max_e p(e)p(f|e) decoding manually #TODO how???
# calculate p(e|f)

------
p( ['the'] | ['das'] ) = 1.0
p( ['the', 'the', 'house', 'a', 'book'] | ['das', 'ein', 'Buch', 'Haus'] ) = 0.0011531999841871323
p( ['book', 'a'] | ['Buch', 'ein'] ) = 0.24999999997398636
p( ['book', 'the'] | ['Buch', 'ein'] ) = 2.601366715804536e-11
p( ['kgefw', 'fek'] | ['Buch', 'ein'] ) = 0.0
IBM Model 1 : 40 % correct
IBM Model 2 : 0 % correct


In [3]:
# Phrase - based MT

def word_alignment(e, f, t_e2f, a_e2f, t_f2e, a_f2e):
    all_a = {}
    for si in range(0, len(f)):
        e2f = viterbi_alignment(e[si], f[si], t_e2f, a_e2f)
        f2e = viterbi_alignment(f[si], e[si], t_f2e, a_f2e)
        print(e[si],f[si])
        print('e2f',e2f)
        print('f2e',f2e)
        a = combine(f2e, e2f)
        print('combine', a)
        print('compare aligns',a == all_alignments_test[si], a, all_alignments_test[si])
        #TODO which behaviour do I want?
        #case word has never been seen before > 
        #case word doesn't have probability with other word >
        all_a[si] = a
        print('-----')
    return all_a

# IBM Model 2 for word alignment
# TODO actually use e_train but here e_test only to test
all_a_test = word_alignment(e_test, f_test, t_e2f, a_e2f, t_f2e, a_f2e)
#In my first approach I had implemented the word alignment using the alignment matrices as in the lecture. That turned out as a reason for unnecessary big memory usage and so I decided instead to use a dict as a datastructure with a mapping from index to list of indices.

['the'] ['das']
e2f {0: 0}
f2e {0: 0}
combine {0: [0]}
compare aligns True {0: [0]} {0: [0]}
-----
['the', 'the', 'house', 'a', 'book'] ['das', 'ein', 'Buch', 'Haus']
e2f {0: 0, 1: 0, 2: 3, 3: 1, 4: 2}
f2e {0: 0, 1: 3, 2: 4, 3: 2}
combine {0: [0], 1: [3], 2: [4], 3: [2]}
compare aligns False {0: [0], 1: [3], 2: [4], 3: [2]} {0: [0, 1], 1: [3], 2: [4], 3: [2]}
-----
['book', 'a'] ['Buch', 'ein']
e2f {0: 0, 1: 1}
f2e {0: 0, 1: 1}
combine {0: [0], 1: [1]}
compare aligns True {0: [0], 1: [1]} {0: [0], 1: [1]}
-----
['book', 'the'] ['Buch', 'ein']
e2f {0: 0, 1: 0}
f2e {0: 0, 1: 0}
combine {0: [0], 1: []}
compare aligns False {0: [0], 1: []} {0: [0]}
-----
['kgefw', 'fek'] ['Buch', 'ein']
e2f {0: None, 1: None}
f2e {0: None, 1: None}
combine {}
compare aligns True {} {}
-----


In [12]:
"""
e_train = [['michael','assumes','that','he','will','stay','in','the','house']]
f_train = [['michael','geht','davon','aus',',','dass','er','im','haus','bleibt']]
all_a = {0:{0:[0],1:[1,2,3],2:[5],3:[6],4:[9],5:[9],6:[7],7:[7],8:[8]}}
# all_a[a][b] alignment for b word in e for sentence a
"""

#TODO do for train
#all_BP = phrase_extraction(e_train, f_train, all_a_train)
all_BP = phrase_extraction(e_test, f_test, all_a_test)

# probabilistic phrase translation table
# For each sentence pair, we extract a number of phrase pairs.
# Then, we count in how many sentence pairs a particular phrase pair is
# extracted and store this number in count(e,f).
counts = {}
for (a, b) in all_BP:
	# key = (frozenset(a),frozenset(b))
	key = str((a, b))
	counts[key] = counts.get(key, 0) + 1
print(counts)

#TODO phrase translation table
print(φ(['house'], ['Haus'], counts))
print(φ(['the','house'],['das','Haus'],counts))

#bigram language model
unigram_counts, bigram_counts = count_grams(e_test)
prev = 'the'
cur = 'book'
#print(bigram_prob(prev, cur, unigram_counts, bigram_counts))
cur = 'a'
#print(bigram_prob(prev, cur, unigram_counts, bigram_counts))

# TODO phrase-based statistical machine translation model
# • the phrase translation table φ(f |e);
# • the reordering model d;
# • the language model pLM(e).
# e_best = argmax_e prod_i φ(f|e) * d(start_i - end_i_min_1 - 1) * prod_i PLM(e_i|e_i_min_1)
for sentence in range(0, len(all_alignments_test)):
    f = f_test[sentence]
    e = e_test[sentence]
    alignment = all_alignments_test[sentence]
    max_value = 0 
    e_best = None
    for j in range(0, len(e)):
        e_j = e[j]
        start_j, end_j_min_1 = calc_vals_for_d(alignment, j, f)
        phrase_transl = 1
        PLM = 1
        for i in range(0, len(f)):
            #phrase_transl *= φ(e_j,f[i], counts) #TODO always 0
            if j > 0: prev = e[j-1] #TODO wrong, I dont get it
            else: prev = e[j]
            PLM *= bigram_prob(prev, e_j, unigram_counts, bigram_counts)
        value = phrase_transl * d(start_j - end_j_min_1 - 1) * PLM
        if value > max_value:
            max_value = value 
            e_best = e_j
    print(e_best, max_value)
            
        #print(j, start_j, end_j_min_1, start_j - end_j_min_1 - 1)  #to test reordering
#todo implement max_phrase_length

{"(['the'], ['das'])": 2, "(['the'], ['das', 'ein'])": 1, "(['the', 'the', 'house', 'a'], ['das', 'ein', 'Buch', 'Haus'])": 1, "(['the', 'the', 'house', 'a', 'book'], ['das', 'ein', 'Buch', 'Haus'])": 1, "(['the'], ['Haus'])": 1, "(['the', 'house'], ['Haus'])": 1, "(['the', 'house', 'a'], ['Buch', 'Haus'])": 1, "(['the', 'house', 'a'], ['ein', 'Buch', 'Haus'])": 1, "(['the', 'house', 'a', 'book'], ['Buch', 'Haus'])": 1, "(['the', 'house', 'a', 'book'], ['ein', 'Buch', 'Haus'])": 1, "(['house', 'a'], ['Buch'])": 1, "(['house', 'a'], ['ein', 'Buch'])": 1, "(['house', 'a', 'book'], ['Buch'])": 1, "(['house', 'a', 'book'], ['ein', 'Buch'])": 1, "(['a'], ['Buch'])": 1, "(['a'], ['ein', 'Buch'])": 1, "(['a', 'book'], ['Buch'])": 1, "(['a', 'book'], ['ein', 'Buch'])": 1, "(['book'], ['Buch'])": 2, "(['book', 'a'], ['Buch', 'ein'])": 1, "(['a'], ['ein'])": 1, "(['book'], ['Buch', 'ein'])": 1, "(['book', 'the'], ['Buch'])": 1, "(['book', 'the'], ['Buch', 'ein'])": 1}
0
0
the 0.2
book 0.00195312

In [5]:
#TODOS

#do summary 09!

#TODO unaligned cases!

#TODO wrong, use start and end! in counts!

#debug ibm 2
#IBM Model 2: How to calculate a correctly?
#my a is 1 when it should be low :(
#implementation exactly like in book
#eg sum is supposed to always be 1
#use ibm model 2 in slides
#compare subresults if they are right, count, stotal etc
#look into indexing, starting 0 / 1?

# print(φ(['house'], ['Haus'], count))
# print(φ(['the','house'],['das','Haus']))
# phrase translation table doesnt work, always returns 0

# TODO translation probability is not working for phrases containing several words
#p(e|f) for IBM Model 2: Gives result 4 should be 1?
#is sum wrong maybe?
#> compare with other implementations

#calculating p(e|f) phrase based model

#decoding > Nazmi
#generate some example target sentences (word sequences) “manually”
#&gen alignments

#TODO save t, a in files & load them?

#structure code more? classes ML model, so it's intuitive to execute

#calc how long stuff takes
#look into efficiency issues, use hashing?
# TODO For the estimation of the phrase translation probabilities, not all
# phrase pairs have to be loaded into memory. It is possible to efficiently
# estimate the probability distribution by storing and sorting the extracted
# phrases on disk. Similarly, when using the translation table for the translation of a single sentence, only a small fraction of it is needed and may
# be loaded on demand.
# think about saving stuff in file
# • Phrase translation table typically bigger than corpus
# ... even with limits on phrase lengths (e.g., max 7 words)
# → Too big to store in memory?
# • Solution for training
# – extract to disk, sort, construct for one source phrase at a time
# • Solutions for decoding
# – on-disk data structures with index for quick look-ups
# – suffix arrays to create phrase pairs on demand
# TODO use numpy for everything
# maybe other data structure ['the','house']:[['das','Haus'],['ein','Haus']]
# faster sometimes but slower if translation in other direction

In [15]:
####### IBM MODELS #######

# read in data
nr_used_sentences = 1800
path = "./src/Paralel Corpus/"
e, f = read_in_corpus(nr_used_sentences, path)

training_split = 0.9
e_train, e_test = split_dataset(e, training_split)
f_train, f_test = split_dataset(f, training_split)
    
max_steps = 25

# execute IBM Model 1 and 2 each two times (English-Turkish and Turkish-English)
t_e2f_ibm1 = EM_IBM_Model_1(e_train, f_train, max_steps)
t_e2f, a_e2f = EM_IBM_Model_2(e_train, f_train, t_e2f_ibm1, max_steps)
t_f2e_ibm1 = EM_IBM_Model_1(f_train, e_train, max_steps)
t_f2e, a_f2e = EM_IBM_Model_2(f_train, e_train, t_f2e_ibm1, max_steps)

# testing
''''
for i in range(0, len(f_test)):
    f = f_test[i]
    e = e_test[i]
    print(e)
    p1 = prob_e_given_f_1(e, f, 1, t_f2e_ibm1)
    print('IBM Model 1 :p(', e, '|', f, ') =', p1)
    p2 = prob_e_given_f_2(e, f, 1, t_f2e, a_f2e)
    print('IBM Model 2: p(', e, '|', f, ') =', p2)
''''
examples_f = [['adam', 'bunu', 'anladı', 'sigarasını', 'söndürdü', 'kadının', 'elini', 'sıkarak', 'hole', 'çıktı']]#,['girdiği', 'bütün', 'işlerde', 'bir', 'terslikle', 'karşılaşmış', 'şimdi', 'de', 'demiryolu', 'işletmesine', 'geçmişti'], ['duruşma', 'aralarında', 'çayım', 'içip', 'sigarasını', 'tüttürerek', 'biraz', 'siyasetten', 'biraz', 'günlük', 'işlerden', 'biraz', 'kâğıt', 'oyunlarından', 'daha', 'çok', 'da', 'atamalardan', 'söz', 'açtığı', 'olurdu']]
examples_e = [['the' ,'man', 'understood','this','and','extinguished','his','cigarette','on','the','womans','hand']]
#He had ruined his prospects in a number of positions and was not serving in the railway department. 
#In the intervals between the sessions he smoked, drank tea, chatted a little about politics, a little about general topics, a little about cards, but most of all about official appointments. 
for i in range(0, len(examples_f)):
    f = examples_f[i]
    e = examples_e[i]
    p1 = prob_e_given_f_1(e, f, 1, t_f2e_ibm1)
    print('IBM Model 1 :p(', e, '|', f, ') =', p1)
    p2 = prob_e_given_f_2(e, f, 1, t_f2e, a_f2e)
    print('IBM Model 2: p(', e, '|', f, ') =', p2)


KeyboardInterrupt: 

In [ ]:
####### PHRASE BASED MODEL #######

# use IBM Model 2 for Viterbi alignments & combine them

all_a_train = {}
for si in range(0, len(f_train)):
    e2f = viterbi_alignment(e_train[si], f_train[si], t_e2f, a_e2f)
    f2e = viterbi_alignment(f_train[si], e_train[si], t_f2e, a_f2e)
    a = combine(f2e, e2f)
    all_a_train[si] = a

# extract phrases & estimate phrase translation probabilities

max_phrase_len = 5 #TODO
all_BP = phrase_extraction(e_train, f_train, all_a_train, max_phrase_len)
phrase_counts = count_phrases(all_BP)

# train bigram language model LM

unigram_counts, bigram_counts = count_grams(e_train)

# testing

examples_f = [['adam', 'bunu', 'anladı', 'sigarasını', 'söndürdü', 'kadının', 'elini', 'sıkarak', 'hole', 'çıktı']]#,['girdiği', 'bütün', 'işlerde', 'bir', 'terslikle', 'karşılaşmış', 'şimdi', 'de', 'demiryolu', 'işletmesine', 'geçmişti'], ['duruşma', 'aralarında', 'çayım', 'içip', 'sigarasını', 'tüttürerek', 'biraz', 'siyasetten', 'biraz', 'günlük', 'işlerden', 'biraz', 'kâğıt', 'oyunlarından', 'daha', 'çok', 'da', 'atamalardan', 'söz', 'açtığı', 'olurdu']]
examples_e = [['the' ,'man', 'understood','this', 'and','extinguished','his','cigarette','leaving','a','hole','on','the','womans','hand']]
#He had ruined his prospects in a number of positions and was not serving in the railway department. 
#In the intervals between the sessions he smoked, drank tea, chatted a little about politics, a little about general topics, a little about cards, but most of all about official appointments. 
examples_a = [[0:[0], 1:[0], 2:[2], 3:[1], 4:[None], 5:[4], 6:[3], 7:[3], 8:[9], 9:[8], 10:[8], 11:[6], 12:[5], 13:[5], 14:[6]]
for i in range(0, len(examples_f)):
    f = examples_f[i]
    e = examples_e[i]
    a = examples_a[i]
    prob = prob_e_given_f(e, f, a)
    print('p(', e, '|', f, ') =', prob)
              
# e_best = argmax_e prod_i φ(f|e) * d(start_i - end_i_min_1 - 1) * prod_i PLM(e_i|e_i_min_1)
def prob_e_given_f(e, f, a, phrase_counts, unigram_counts, bigram_counts)
    max_value = 0
    e_best = None
    for j in range(0, len(e)):
        e_j = e[j]
        #TODO how to calc prob for sentence / how to for word?
        PT_prob = PT_prob(e_j, f[i], phrase_counts)
        prev = 0 #TODO wrong, use start and end! in counts!
        LM_prob = LM_prob(prev, e_j, unigram_counts, bigram_counts)
        start_j, end_j_min_1 = calc_vals_for_d(a, j, f)
        d = d(start_j - end_j_min_1 - 1)
        prob = PT_prob * d * LM_prob
    return prob